# Gradient Boosting
Im Gegensatz zum vorherigen Notebook liegt der Fokus hier weniger auf Interpretierbarkeit, sondern auf einer möglichst hohen Modellperformance. Dafür verwende ich **Gradient Boosting**, genauer gesagt, eignet sich für unsere Daten insbesondere CatBoost, ein Boosting Modell, welches insbesondere gut mit hoch-kardinalen Features umgehen kann.

## Gliederung

#### 1. Daten laden & Überblick
- Importieren der benötigten Bibliotheken
- Laden des vorbereiteten Datensatzes
- Kurzer Blick auf den Datensatz

#### 2. Baseline-Modell
- Definition einer Funktion für CatBoost
- Baseline ohne Modifikationen

#### 3. Feature Engineering
- Modifikationen aus Notebook 3
- Zusätzliche Interaktionsfeatures

#### 4. Tuning mit Optuna
- Train/Valid-Split + Pooling
- Erstellen einer Objective Function
- Durchführung der Studie und Anwendung der Hyperparameter

#### 5. Vergleich und Fazit
- Vergleich der Modelle
- Fazit

## Import
#### Importieren der benötigten Bibliotheken

In [3]:
!pip install catboost
!pip -q install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 8.2 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
from pathlib import Path

from catboost import CatBoostRegressor, Pool
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split, cross_val_score
import optuna

np.random.seed(42)
optuna.logging.set_verbosity(optuna.logging.WARNING)

#### Laden des vorbereiteten Datensatzes + Kurzer Blick

In [9]:
data_dir = Path('data/processed')
data_file = data_dir / 'cleaned_data_02.csv'
df = pd.read_csv(data_file)
df.head()

,fuel_type,kms_run,sale_price,city,body_type,transmission,variant,registered_city,registered_state,rto,make,model,total_owners,car_rating,fitness_certificate,warranty_avail,age
0,petrol,8063,386399,noida,hatchback,manual,lxi opt,delhi,delhi,dl6c,maruti,swift,2,great,True,False,6
1,petrol,23104,265499,noida,hatchback,manual,lxi,noida,uttar pradesh,up16,maruti,alto 800,1,great,True,False,5
2,petrol,23402,477699,noida,hatchback,manual,sports 1.2 vtvt,agra,uttar pradesh,up80,hyundai,grand i10,1,great,True,False,4
3,diesel,39124,307999,noida,hatchback,manual,vdi,delhi,delhi,dl1c,maruti,swift,1,great,True,False,8
4,petrol,22116,361499,noida,hatchback,manual,magna 1.2 vtvt,new delhi,delhi,dl12,hyundai,grand i10,1,great,True,False,6


## Baseline

Mit der Ridge-Regression aus dem vorherigen Notebook gelang uns ein **R2** von **0.9044** und ein **MAE** von **45783**. Sehen wir zunächst was CatBoost erzielt, wenn wir keine weiteren Änderungen an Daten oder Hyperparametern vornehmen.

In [10]:
def get_cb_cv(data, cv = 5, params = None, return_mae = False):
    if params is None:
        params = {}
    X = data.copy()
    y = X.pop('sale_price')

    cat_idx = np.where(X.dtypes == 'object')[0]

    model = CatBoostRegressor(verbose = False, cat_features = cat_idx, loss_function = 'RMSE', **params)
    score_r2 = cross_val_score(model, X, y, cv = cv)
    if return_mae:
        score_mae = cross_val_score(model, X, y, cv = cv, scoring = 'neg_mean_absolute_error')
        score_mae = - score_mae
        return score_r2.mean(), score_mae.mean()
    return score_r2.mean()

In [11]:
result_1 = get_cb_cv(df, return_mae = True)
result_1

(np.float64(0.9011385029037307), np.float64(45529.654208410386))

Wendet man CatBoost ohne weitere Modifikationen auf unsere Daten an, so performt es lediglich geringfügig schlechter als das finale Modell von Notebook 3.

## Feature Engineering

In [12]:
#Kopie des Datensatzes. Änderungen werden auf der Kopie durchgeführt.
df_fe = df.copy()

#### Modifikationen aus Notebook 3
Übernehmen wir also zunächst die Modifikationen aus dem vorherigen Notebook, wobei auf Binning verzichtet werden kann, da es bei CatBoost eher negative Effekte hat.

In [13]:
#Zerlegen von rto
df_fe['rto_state'] = df_fe['rto'].astype(str).str[:2]
df_fe['rto_number'] = df_fe['rto'].astype(str).str[2:]

#car_rating numerisch
df_fe['car_rating_number'] = df_fe.car_rating.map({'overpriced' : 1, 'fair' : 2, 'good' : 3, 'great' : 4})

#Interaktionsfeatures
#KM pro Besitzer
df_fe['kms_per_owner'] = df_fe['kms_run'] / df_fe['total_owners']

#KM pro Jahr/ Wie viel wird Auto gefahren?
df_fe['kms_per_age'] = np.where(df_fe['age'] == 0, df_fe['kms_run'], df_fe['kms_run'] / df_fe['age'])

#Durchschnittliche KM pro Modell
df_fe['avg_kms_by_model'] = df_fe.groupby('model')['kms_run'].transform('mean')

#KM im Vergleich zu anderen Autos des gleichen Modells
df_fe['kms_vs_model_avg'] = df_fe['kms_run'] / df_fe['avg_kms_by_model']

# Durchschnittliches Alter pro Modell
df_fe['avg_age_by_model'] = df_fe.groupby('model')['age'].transform('mean')

#Alter im Vergleich zu anderen Autos des gleichen Modells
df_fe['age_vs_model_avg'] = df_fe['age'] / df_fe['avg_age_by_model']

#Druchschnittliches Rating pro Modell
df_fe['avg_rating_by_model'] = df_fe.groupby('model')['car_rating_number'].transform('mean')

#Rating im Vergleich zu anderen Autos des gleichen Modells
df_fe['rating_vs_model_avg'] = df_fe['car_rating_number'] / df_fe['avg_rating_by_model']

In [14]:
df.head()

,fuel_type,kms_run,sale_price,city,body_type,transmission,variant,registered_city,registered_state,rto,make,model,total_owners,car_rating,fitness_certificate,warranty_avail,age
0,petrol,8063,386399,noida,hatchback,manual,lxi opt,delhi,delhi,dl6c,maruti,swift,2,great,True,False,6
1,petrol,23104,265499,noida,hatchback,manual,lxi,noida,uttar pradesh,up16,maruti,alto 800,1,great,True,False,5
2,petrol,23402,477699,noida,hatchback,manual,sports 1.2 vtvt,agra,uttar pradesh,up80,hyundai,grand i10,1,great,True,False,4
3,diesel,39124,307999,noida,hatchback,manual,vdi,delhi,delhi,dl1c,maruti,swift,1,great,True,False,8
4,petrol,22116,361499,noida,hatchback,manual,magna 1.2 vtvt,new delhi,delhi,dl12,hyundai,grand i10,1,great,True,False,6


In [ ]:
get_cb_cv(df_fe)

0.9115901413573338

Mit den neuen Features ist CatBoost bereits deutlich performanter als unsere Ridge-Regression.

#### Zusätzliche Interaktionsfeatures
CatBoost ist grundsätzlich robust gegenüber überflüssigen Features und führt durch seine baumbasierte Struktur eine implizite Feature-Selektion während des Trainings durch.
Im Boosting-Verfahren werden sukzessive Modelle auf den Residuen der vorherigen Modelle trainiert, sodass Merkmale ohne relevanten Informationsgehalt nur selten für Splits verwendet werden.

Dennoch ist es sinnvoll, unnötige Features zu vermeiden, da sie den Rechenaufwand erhöhen und in Extremfällen die Generalisierung beeinträchtigen können.
Im vorliegenden Datensatz entsteht der überwiegende Teil der Modellkomplexität jedoch durch hoch-kardinale kategoriale Merkmale wie **variant** und **model**.
Die zusätzlich erzeugten numerischen Ratio-Features sind rechnerisch günstig und fallen im Vergleich dazu kaum ins Gewicht.

Vor diesem Hintergrund ist es sinnvoll, dem Modell gezielt weitere niedrigdimensionale Interaktionen bereitzustellen, die explizite, domänenspezifische Signale enthalten.

In [ ]:
#Indikator für Verschleiß KM * Age
df_fe['kms_times_age'] = df_fe['kms_run'] * df_fe['age']

#Häufigkeit des jeweiligen Modells
df_fe["model_frequency"] = df_fe.groupby("model")["model"].transform("count")

#Häufigkeit der jeweiligen Variante
df_fe["variant_frequency"] = (df_fe.groupby(["model", "variant"])["variant"].transform("count"))

In [ ]:
result_2 = get_cb_cv(df_fe, return_mae = True)
result_2

(0.9134306542490481, 41050.19946511569)


Die zusätzlichen Features erhöhen tatsächlich weiter die Modellperformance.

## Tuning
Während bei der Ridge Regression lediglich ein einzelner Regularisierungsparameter zu bestimmen war, erfordert CatBoost die Abstimmung einer größeren Anzahl an Hyperparametern, die Modellkomplexität, Regularisierung und mehr steuern.

Das Hyperparameter-Tuning wurde mithilfe von Optuna durchgeführt. Anstelle einer vollständigen Cross-Validation wurde ein Hold-out-Validierungsansatz gewählt, da das Training eines einzelnen CatBoost-Modells aufgrund der hoch-kardinalen Features bereits vergleichsweise rechenintensiv ist.

Optimiert wurden zentrale Parameter wie Baumtiefe, Lernrate und Regularisierungsterm (l2_leaf_reg).

#### Train/Valid-split + Pooling

In [ ]:
X = df_fe.drop(columns=['sale_price'])
y = df_fe['sale_price']

#Aufteilen in Trainingsund Validierungsdaten
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42
)

#Markieren der kategorischen Features für CatBoost
cat_features = X_train.select_dtypes('object').columns.tolist()

#Erstellen von Train und Valid-Pools
train_pool = Pool(X_train, y_train, cat_features=cat_features)
valid_pool = Pool(X_valid, y_valid, cat_features=cat_features)


#### Erstellen einer Objective-Function
Die Objective Function trainiert für jede von Optuna vorgeschlagene Hyperparameter-Konfiguration ein CatBoost-Modell und bewertet dessen Performance.
Der Fokus liegt auf zentralen Hyperparametern der Modellkomplexität und Regularisierung, während Early Stopping eine automatische Begrenzung der effektiven Modellgröße sicherstellt.

In [ ]:
def objective(trial):
    params = {
    "depth": trial.suggest_int("depth", 4, 10),
    "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
    "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-2, 10.0, log=True),
    "iterations": 500,
    "loss_function": "RMSE",
    "eval_metric": "RMSE",
    "verbose": False,
    "random_seed": 42
    }

    model = CatBoostRegressor(**params)

    model.fit(
        train_pool,
        eval_set=valid_pool,
        use_best_model=True,
        early_stopping_rounds=100,
    )

    y_pred = model.predict(valid_pool)
    score = r2_score(y_valid, y_pred)

    return score


#### Durchführen der Studie und Anwendung der Hyperparameter

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=40)

print("Best R2:", study.best_value)
print("Best params:")
study.best_params

Best R2: 0.9401777360406103
Best params:


{'depth': 5,
 'learning_rate': 0.16375340470386598,
 'l2_leaf_reg': 0.03214081097281057}

In [ ]:
params = {'depth': 7,
 'learning_rate': 0.07100301167871348,
 'l2_leaf_reg': 0.16105080712816852}

In [ ]:
#Robusteres Ergebnis mit cv = 10
result_3 = get_cb_cv(df_fe, cv = 10, params = params, return_mae = True)
result_3

(0.9242117783619482, 38959.04453879014)

## Vergleich und Fazit
#### Vergleicht der Modelle
Für einen besseren Überblick stellen wir die Modellperformance der einzelnen Schritte aufbereitet in einem DataFrame gegenüber.

In [ ]:
result_0 = (0.9043663582952732, 45783.12850969297)
steps = [
    ('Ridge-Regression Notebook 3', result_0),
    ('CatBoost ohne Modifikationen', result_1),
    ('CatBoost mit neuen Features', result_2),
    ('CatBoost nach Tuning', result_3)
]

rows = []
for name, (r2, mae) in steps:
    rows.append((name, round(r2, 4), round(mae, 0)))

df_results = pd.DataFrame(rows, columns = ['Modellschritt', 'R2 (CV)', 'MAE (CV)'])
df_results.index = df_results.index + 1
df_results

,Modellschritt,R2 (CV),MAE (CV)
1,Ridge-Regression Notebook 3,0.9044,45783.0
2,CatBoost ohne Modifikationen,0.9011,45530.0
3,CatBoost mit neuen Features,0.9134,41050.0
4,CatBoost nach Tuning,0.9242,38959.0


#### Fazit
Im Vergleich zu den linearen Modellen erzielt CatBoost eine deutliche Leistungssteigerung, insbesondere durch den effektiven Umgang mit hoch-kardinalen kategorialen Features.
Feature Engineering aus vorherigen Analysen konnte die Modellgüte weiter verbessern, während zusätzliche Interaktionsfeatures keinen zusätzlichen Mehrwert lieferten.

Das finale, getunte CatBoost-Modell stellt einen guten Kompromiss zwischen Modellkomplexität, Rechenaufwand und Vorhersagegüte dar.
Insgesamt zeigt sich, dass leistungsfähige Boosting-Modelle viele nichtlineare Zusammenhänge bereits implizit erfassen und zusätzliche Feature-Komplexität sorgfältig abgewogen werden sollte.